In [95]:
import numpy as np
import pandas as pd
import re

In [155]:
stock_name = "AMZN"

# clean message data

In [208]:
df = pd.read_csv('data/AMZN_2012-06-21_34200000_57600000_message_5.csv')
df.columns = ["time", "event type lobster", "order id", "size", "price", "direction"]
relevant_rows_bool = df["event type lobster"].isin([1,3])
df = df[relevant_rows_bool].reset_index(drop=True)

In [209]:
df['event type thesis'] = None

In [210]:
df['event type thesis'][(df['direction'] == -1 ) & (df['event type lobster'] == 1)] = 2
df['event type thesis'][(df['direction'] == 1 ) & (df['event type lobster'] == 1)] = 0
df['event type thesis'][(df['direction'] == -1 ) & (df['event type lobster'] == 3)] = 3
df['event type thesis'][(df['direction'] == 1 ) & (df['event type lobster'] == 3)] = 1

C:\Users\jobei\AppData\Local\Temp\ipykernel_17040\1620800425.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['event type thesis'][(df['direction'] == -1 ) & (df['event type lobster'] == 1)] = 2
C:\Users\jobei\AppData\Local\Temp\ipykernel_17040\1620800425.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['event type thesis'][(df['direction'] == 1 ) & (df['event type lobster'] == 1)] = 0
C:\Users\jobei\AppData\Local\Temp\ipykernel_17040\1620800425.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pa

In [212]:
df

,time,event type lobster,order id,size,price,direction,event type thesis
0,34200.189608,1,11885113,21,2238100,1,0
1,34200.189608,1,3911376,20,2239600,-1,2
2,34200.189608,1,11534792,100,2237500,1,0
3,34200.189608,1,1365373,13,2240000,-1,2
4,34200.189608,1,11474176,2,2236500,1,0
...,...,...,...,...,...,...,...
144061,57599.545699,3,287094952,60,2206400,-1,3
144062,57599.903989,3,287142900,100,2206200,-1,3
144063,57599.955242,3,286967592,170,2206900,-1,3
144064,57599.958245,1,287174077,100,2206300,-1,2


In [211]:
df.to_csv(f"data/clean/{stock_name}_message_data.csv")

# Clean orderbook data

In [199]:
theta = 1/3

In [200]:
df = pd.read_csv('data/AMZN_2012-06-21_34200000_57600000_orderbook_5.csv')

In [201]:
df.columns = ['ask price 1', 'ask size 1', 'bid price 1', 'bid size 1',
             'ask price 2', 'ask size 2', 'bid price 2', 'bid size 2',
             'ask price 3', 'ask size 3', 'bid price 3', 'bid size 3',
             'ask price 4', 'ask size 4', 'bid price 4', 'bid size 4',
             'ask price 5', 'ask size 5', 'bid price 5', 'bid size 5',]

In [202]:
state_indicator = (df['bid size 1'] - df['ask size 1'])/(df['bid size 1'] + df['ask size 1'] + 0.0001) #decimal number to keep set "open" and not include -1 and 1
df['state indicator'] = np.floor((state_indicator + 1)* (3/2))

In [203]:
df = df[relevant_rows_bool].reset_index(drop = True)

In [204]:
df.to_csv(f"data/clean/{stock_name}_orderbook_data.csv")

# transition matrix

In [376]:
def get_matrix_pos(series):
    return((int(series[0]), int(series[1])))

In [377]:
df_message = pd.read_csv(f'data/clean/{stock_name}_message_data.csv', index_col=0)
df_book = pd.read_csv(f'data/clean/{stock_name}_orderbook_data.csv', index_col=0)

In [378]:
df_book['after event state indicator'] = df_book['state indicator'].shift(-1)
df_book = df_book.dropna()

In [379]:
transition_matrix_coordinates = df_book[["state indicator", 'after event state indicator']].apply(get_matrix_pos, axis=1)
transition_matrix_index = df_message['event type thesis'][:-1]

In [382]:
transition_matrix = np.zeros((4,3,3))
for index, coordinate in zip(transition_matrix_index, transition_matrix_coordinates):
    transition_matrix[index][coordinate] += 1

In [383]:
factors = transition_matrix.sum(axis=(2))

In [384]:
for i,(matrix, factor) in enumerate(zip(transition_matrix, factors)):
    transition_matrix[i] = matrix / factor.reshape(-1,1)

In [390]:
np.save(f"data/transition_matrices/{stock_name}_matrix.npy", transition_matrix)